In [4]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32'
import time
import sys, getopt
import numpy as NP
import theano
import theano.tensor as T
import theano.tensor.nnet as NN
from wrapper import *
from collections import OrderedDict
from txt_data_word import *

gru2_dim = 200 # gru in word LM
emb_dim = 100 # word emb dim, must equal to gru1_dim due to the high way network
batch_size = 1
drop_flag = False
train_flag = True
mode = None
fname = 'ptb_vis'
Dataset = txt_data('ptb.train', 'ptb.valid', 'ptb_dict', ['ptb.train', 'ptb.valid', 'ptb.test'], batch_size=20)

word_dim = Dataset.w_dim # vocabulary size

var_lr = theano.shared(NP.asarray(1.0, dtype=theano.config.floatX)) #learning rate of rmsprop
var_rescale = theano.shared(NP.asarray(5.0, dtype=theano.config.floatX)) # rescale the gradient norm
word_in = T.imatrix()
word_target = T.imatrix()
model = Model()

#get the mask for handling various length in the batch, the all zeros vector would be masked out
def get_mask(x_in, mask_value=0.):
    return T.shape_padright(T.any((1. - T.eq(x_in, mask_value)), axis=-1))

def _word_rnn(x, prev_h):
    gru1 = model.gru(x, prev_h, name='_word_rnn', shape=(emb_dim, gru2_dim))
    return gru1


drop_flag = False
EPSI = 1e-15

train_state_h1 = theano.shared(NP.zeros((batch_size , gru2_dim), dtype=theano.config.floatX))

test_state_h1 = theano.shared(NP.zeros((batch_size , gru2_dim), dtype=theano.config.floatX))

def get_express(train=False, emb_flag=None):
        global drop_flag
        drop_flag = train
        state_updates = []
        state_h1 = train_state_h1 if train else test_state_h1
        word_embs = model.embedding(word_in, name = 'word_emb', shape = (word_dim, emb_dim))
        
        seq_len = word_in.shape[1]
        sc, _ = theano.scan(_word_rnn, sequences=[word_embs.dimshuffle(1,0,2)], outputs_info=[state_h1])
        word_out = sc.dimshuffle(1,0,2)
        state_updates.append((state_h1, sc[-1]))
        if drop_flag:
            word_out = drop5.drop(word_out)

        word_out = word_out.reshape((word_out.shape[0]*(word_out.shape[1]), gru2_dim))

        word_out = model.h_softmax(x_in = word_out, name='word_hs', shape=(gru2_dim, word_dim))
        
        return word_out[-1]

wo = get_express(train=False, emb_flag=mode)
predict_func = theano.function([word_in], [wo], allow_input_downcast=True)

model.load('ptb_vis_19')
print 'TEST COMPILE'
    

def reset_states():
    test_state_h1.set_value(test_state_h1.get_value()*0.0)


(1327, 20, 35, 25)
(1327, 20, 35)
(1327, 20, 35)
Data Size  10201
word_emb {'shape': (10201, 100)}
_word_rnn {'shape': (100, 200)}
word_hs {'shape': (200, 10201)}
H_softmax, two level  101 101
TEST COMPILE


In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy.random as RNG
Wdict = Dataset.Wdict
num2w = Dataset.num2w
num2w[0]='<unk>'
st=['it'] # given the start words

reset_states()

for i in xrange(10): # the sentence length
    w_in = []
    for j in st:
        w_in.append(Wdict[j])
    t = predict_func(NP.asarray([w_in]))[0] # generating the next word
    next_idx=RNG.choice(word_dim, 1, p=t)[0] # sampling
    next_word = num2w[next_idx]
    st.append(next_word)

for i in xrange(len(st)):
    if st[i] ==u'\u1d94':
        st[i] = '<unk>'
    if st[i] ==u'\u1d93':
        st[i] = '<eos>'
print st # generated sentence 

['it', u'subject', u'will', u'normally', u'a', u'party', u'of', u'seven', '<unk>', u'payments', '<eos>']


In [ ]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32'
import time
import sys, getopt
import numpy as NP
import theano
import theano.tensor as T
import theano.tensor.nnet as NN
from wrapper import *
from collections import OrderedDict
from txt_data_word import *

class Moving_AVG(object):
	def __init__(self, array_size=500):
		self.array_size=array_size
		self.queue = NP.zeros((array_size, ))
		self.idx = 0
		self.filled = False
	def append(self, x):
		self.queue[self.idx]=x
		self.idx = (self.idx + 1)%self.array_size
		if not self.filled and self.idx==0:
			self.filled = True
	def get_avg(self):
		result = NP.mean(self.queue) if self.filled else NP.sum(self.queue)/NP.sum(self.queue!=0)
		return result



gru2_dim = 200 # gru in word LM
emb_dim = 100 # word emb dim, must equal to gru1_dim due to the high way network
char_emb_dim = 15
batch_size = 20
drop_flag = False
RHO=0.4
half_W = 6
FREQ_T = 1e-3
train_flag = True
mode = None
fname = 'ptb_vis'
Dataset = txt_data('ptb.train', 'ptb.valid', 'ptb_dict', ['ptb.train', 'ptb.valid', 'ptb.test'], batch_size=20)

word_dim = Dataset.w_dim # word dict size

var_lr = theano.shared(NP.asarray(1.0, dtype=theano.config.floatX)) #learning rate of rmsprop
var_rescale = theano.shared(NP.asarray(5.0, dtype=theano.config.floatX)) # rescale the gradient norm
word_in = T.imatrix()
word_target = T.imatrix()
model = Model()
drop2 = Dropout(shape=(word_target.shape[0]*word_target.shape[1], emb_dim), prob=0.0)
drop3 = Dropout(shape=(word_target.shape[0], gru2_dim), prob=0.0)
drop4 = Dropout(shape=(word_target.shape[0], emb_dim), prob=0.0)
drop5 = Dropout(shape=(word_target.shape[0], word_target.shape[1], gru2_dim), prob=0.5)
drop6 = Dropout(shape=(word_target.shape[0], word_target.shape[1], gru2_dim), prob=0.0)

def categorical_crossentropy(prob, true_idx):
    true_idx = T.arange(true_idx.shape[0]) * word_dim + true_idx
    t1 = prob.flatten()[true_idx]
    return -t1

#get mask for handle variable length in batch, all zeros vector will be masked out
def get_mask(x_in, mask_value=0.):
    return T.shape_padright(T.any((1. - T.eq(x_in, mask_value)), axis=-1))

#character step

def letanh(x):
    return 1.7159*tanh(2.0/3.0*x)

def relu(x):
    return T.switch(x>0, x, 0.1*x)

def high_way(x_in=None, name='', shape=[]):
	g = sigmoid(model.fc(x_in = x_in, name=name+'_g', shape=shape))
	h = relu(model.fc(x_in=x_in, name=name+'_h', shape=shape))
	return h*g + x_in * (1. - g)

def softmax(x):
    e_x = T.exp(x)
    sm = e_x / e_x.sum(axis=1, keepdims=True)
    return sm

def softmax_t(x, t):
    e_x = T.exp(1.0*x/t)
    sm = e_x / e_x.sum(axis=1, keepdims=True)
    return sm

def log_softmax(x):
    xdev = x - x.max(1, keepdims=True)
    return xdev - T.log(T.sum(T.exp(xdev), axis=1, keepdims=True))

def re_sigmoid(x):
    sx = sigmoid(x)
    return T.switch(sx>0.5, sx, 0.0)

def _word_rnn(x, prev_h):
    gru1 = model.gru(x, prev_h, name='_word_rnn', shape=(emb_dim, gru2_dim))
    return gru1

sc, _ = theano.scan(_word_rnn, sequences=[word_embs.dimshuffle(1,0,2)], /
                            outputs_info=[state_h1])

prev_h = 0
for t in xrange(10):
    gru1[t] = model.gru(x[t], prev_h, name='_word_rnn', shape=(emb_dim, gru2_dim))
    prev_h = gru1[t]
    
drop_flag = False
EPSI = 1e-15

train_state_h1 = theano.shared(NP.zeros((batch_size , gru2_dim), dtype=theano.config.floatX))

test_state_h1 = theano.shared(NP.zeros((batch_size , gru2_dim), dtype=theano.config.floatX))

def get_express(train=False, emb_flag=None):
        global drop_flag
        drop_flag = train
        state_updates = []
        state_h1 = train_state_h1 if train else test_state_h1
        word_embs = model.embedding(word_in, name = 'word_emb', shape = (word_dim, emb_dim))
        
        seq_len = word_in.shape[1]
        sc, _ = theano.scan(_word_rnn, sequences=[word_embs.dimshuffle(1,0,2)], /
                            outputs_info=[state_h1])
        word_out = sc.dimshuffle(1,0,2)
        state_updates.append((state_h1, sc[-1]))
        if drop_flag:
            word_out = drop5.drop(word_out)

        word_out = word_out.reshape((word_target.shape[0]*(word_target.shape[1]), gru2_dim))
        f_word_target = word_target.reshape((word_target.shape[0]*(word_target.shape[1]), ))

        word_out = model.h_softmax(x_in = word_out, y_in = f_word_target, name='word_hs', /
                                   shape=(gru2_dim, word_dim))
        word_out = -T.log(T.clip(word_out, EPSI, 1.0-EPSI))

        mask_PPL = (1. - T.eq(f_word_target, -1))*word_out
        PPL_word_LM = T.sum(mask_PPL)
        cost_word_LM = PPL_word_LM/T.sum(f_word_target>=0)
        cost_all = cost_word_LM
        if train:
            grad_all, grad_norm = rmsprop(cost_all, model.get_params(), lr=var_lr, /
                                          epsilon=var_lr**2, rescale = var_rescale , ignore_input_disconnect=True, return_norm=True)
            return cost_all, PPL_word_LM, grad_all + state_updates+model.get_updates(), grad_norm
        else:
            return cost_all, PPL_word_LM, state_updates+model.get_updates()

cost_all, PPL, all_updates = get_express(train=False, emb_flag=mode)
test_func_hw = theano.function([word_in, word_target], [cost_all, PPL], updates=all_updates, /
                               allow_input_downcast=True, on_unused_input='ignore')

print 'TEST COMPILE'

if train_flag :
    cost_all, PPL, all_updates, grad_norm = get_express(train=True, emb_flag=mode)
    train_func_hw = theano.function([word_in, word_target], [cost_all, PPL, grad_norm], / 
                                    updates=all_updates, allow_input_downcast=True, on_unused_input='ignore')
    print 'TRAIN COMPILE'       

train_func = None
test_func = None

def reset_states():
    test_state_h1.set_value(test_state_h1.get_value()*0.0)

last_valid_PPL = 2e10
valid_increase_cnt = 0 #if loss on validation set didn't decrease in N steps, change the learning rate as half
reset_states()
for i in xrange(0, 20):
    train_func = train_func_hw if train_flag else None
    test_func = test_func_hw
    if train_flag:
        mytime = time.time()
        train_batchs = int(Dataset.train_size/batch_size)
        ma_cost = Moving_AVG(train_batchs) 
        for j in xrange(train_batchs):
            word, word_label = Dataset.get_batch(only_word=True)
            n_cost, n_ppl, n_norm = train_func(word, word_label)
            ma_cost.append(n_cost)
            if j%100==0:
                print 'Epoch = ', str(i), ' Batch = ', str(j), ' Cost = ', n_cost, ' PPL = ', NP.exp(n_ppl/NP.sum(word_label>=0)), ' AVG Cost = ', ma_cost.get_avg(), 'LEN = ', NP.shape(word_label)[1], 'GRAD Norm = ', n_norm #NP.mean(NP.abs(model.wmatrix(name = 'word_emb_gain', shape=((1,))).get_value()))

            if j%5000==4999:
                model.save(fname+'_every')
        newtime = time.time()
        print 'One Epoch Time = ', newtime-mytime
        mytime = newtime
        model.save(fname+'_'+str(i))
    if not train_flag:
        model.load(fname+'_'+str(i))
    reset_states()
    model.clear_state()
    Dataset.test_data_idx = 0
    Dataset.test_len_idx = 0
    test_wper = []
    test_wcnt = []
    test_batchs = 0
    while test_batchs<Dataset.test_size:
        word, word_label = Dataset.get_batch(test=True, only_word=True)
        n_cost, n_ppl = test_func(word, word_label)
        test_wper.append(n_ppl)
        test_wcnt.append(NP.sum(word_label>=0))
        test_batchs+=len(word_label)
        print ' Test Progress = ', 1.0*test_batchs/Dataset.test_size, 
    valid_PPL = NP.exp(NP.sum(test_wper)/NP.sum(test_wcnt))
    print '\nEpoch = ', str(i), ' Test Word PPL = ', valid_PPL
    if (valid_PPL>150 and valid_PPL+1 > last_valid_PPL) or (valid_PPL>last_valid_PPL):
        valid_increase_cnt += 1
        if valid_increase_cnt>=1:
            print 'change learning rate', var_lr.get_value()*0.5
            var_lr.set_value(var_lr.get_value()*0.5)
            valid_increase_cnt = 0
    last_valid_PPL = valid_PPL